#  <center>Harris Corner Detection
 <p style ="text-align:right">宋正非</p>

## 基础思想

## &emsp; 1.窗口
   
&emsp;&emsp;对局部特征检测来说，基础的方法是滑动窗口在图向上滑动，检测窗口滑动的内容（如所有像素值梯度）或者内容变化情况（如像素密度变化的整体情况），观察窗口捕捉到的信息是否符合局部特征的特点。


## &emsp;2.角点区域特点

&emsp;&emsp;**角点**：边的交点，边的特征是梯度在某一方向上产生突变，那么这个交点应该表现为某一区域内有两个或多个方向上的**边缘性变化**的梯度信息。
  
  **需要强调的是，寻找角点的目标或许是找到一个“点”，但是分析的对象并不能只是一个“点”（在图像处理中，点指的是像素点），而应该是一个区域所有像素点的整体所呈现出来的特征。**
  
&emsp;&emsp;如果用滑动窗口来观察角点区域的话，窗口向多个方向滑动，都能感知到像素密度的强变化（梯度）。①平坦区域：在所有方向没有明显梯度变化。②边缘区域：沿边方向没有明显梯度变化。③角度边缘：在各个方向梯度值有明显变化。

<img src="pic1.jpg" width=500>

## &emsp;3.量化方法
&emsp;&emsp; 需要度量窗口向某方向滑动前后，窗口内整体像素值的变化情况。重点是**窗口移动方向&像素值变化**

**·构建函数：**<p>
<center> $ E(u,v) = {\sum_{x,y}{{w(x,y)} {\left[I(x+u,y+v)-I(x,y)\right]}^{2}}}$<br>
    
其中，w(x,y)为window function，可以规定函数为1 in window，0 outside，也可以取为高斯函数；I：intensity。

**·公式推导：**<p>
一阶泰勒展开：<center> $ f(x+u,y+v) \approx f(x,y)+{u}{f_x(x,y)}+{v}{f_y(x,y)}$<br>

则 
$$
\begin{aligned}
{\sum}{\left[I(x+u,y+v)-I(x,y)\right]}^{2}&\approx {\sum}{\left[I(x,y)+{u}{I_x}+{v}{I_y}-I(x,y)\right]}^{2} \\
  &= {\sum}{{{u}^{2}}{{I_x}^{2}}+{2}{u}{v}{I_x}{I_y}+{{v}^{2}}{{I_y}^{2}}}\\
  &=\sum\left[\begin{array}{ll}u & v\end{array}\right]\left[\begin{array}{cc}I_x^2 & I_x I_y \\ I_x I_y & I_y^2\end{array}\right]\left[\begin{array}{l}u \\ v\end{array}\right]\\
  & =\left[\begin{array}{ll}u & v\end{array}\right]\left(\sum\left[\begin{array}{cc}I_x^2 & I_x I_y \\ I_x I_y & I_y^2\end{array}\right]\right)\left[\begin{array}{l}u \\ v\end{array}\right]\\
\end{aligned}
$$

将E表示为<center>$ E(u, v) \simeq\left[\begin{array}{ll}u & v\end{array}\right] M\left[\begin{array}{l}u \\ v\end{array}\right] $ <p>

其中<center>$M=\sum_{x, y} w(x, y)\left[\begin{array}{cc}I_x^2 & I_x I_y \\ I_x I_y & I_y^2\end{array}\right]$

对于实对称矩阵，可对其进行正交相似对角化，变为
    ${P}{\left[\begin{array}{cc} \lambda_1 & 0 \\ 0 & \lambda_2 \end{array}\right]}{{P}^{-1}}$,${P}^{T}={P}^{-1}$
    
则<center> $$ 
\begin{aligned}
E(u,v) &={\left[\begin{array}{ll}u & v\end{array}\right]}{P}{\left[\begin{array}{cc} \lambda_1 & 0 \\ 0 & \lambda_2 \end{array}\right]}{{P}^{T}}{\left[\begin{array}{l}u \\ v\end{array}\right]}\\
&= {\left[\begin{array}{ll}u' & v'\end{array}\right]}{\left[\begin{array}{cc} \lambda_1 & 0 \\ 0 & \lambda_2 \end{array}\right]}{\left[\begin{array}{l}u' \\ v'\end{array}\right]}\\
&={\lambda_1}{(u')}^{2}+{\lambda_2}{(v')}^{2}\\
&= \frac{{(u')}^{2}}{\frac{1}{\lambda_1}}+\frac{{(v')}^{2}}{\frac{1}{\lambda_2}}
\end{aligned}
$$


<img src="pic2.jpg" width=250>

**·分析椭圆**<p>
·$\lambda_1$、$\lambda_2$大小决定长短轴，R决定椭圆的朝向。<p>
·椭圆长轴对应${(\lambda_{min})}^{-\frac{1}{2}}$，短轴对应${(\lambda_{max})}^{-\frac{1}{2}}$,两个$\lambda$分别反应区域梯度在两个垂直方向上的大小集中分布情况。

<img src="pic3.jpg" width=500>
·如果两个值都大，且相差不大，则“E函数水平切片椭圆”趋近于圆，区域梯度在两个垂直的方向都有较强集中度，所以是角点区域。<p>
·如果其中一个值远大于另一个值，则，则“E函数水平切片椭圆”趋近于线，区域梯度只在一个方向上有较强集中度，所以是边缘区域。<p>
·如果两个值都很小，则“E函数水平切片椭圆”趋近于点，区域梯度信息在两个方向都较弱，所以是像素值平坦区域。<p>

**·评价函数**<p>
$$ det M={\lambda_1}{\lambda_2}\\
trace M=\lambda_1+\lambda_2 $$<p>
&emsp;&emsp; 进一步用M的特征值定义一个函数，这个函数的值恰好可以综合表达$\lambda_1$和$\lambda_2$的三种情况，来简化分类讨论:<p>
$$R={det(M)}-{k}{{trace(M)}^{2}}={\lambda_1}{\lambda_2}-k{(\lambda_1+\lambda_2)}^{2}$$<p>
将M称为“Second moment matrix”，将这个函数R称为“角点响应函数”。<p>
·某个区域的角点响应函数值大于0且绝对值较大时，角点区域。<p>
·某个区域的角点响应函数值小于0且绝对值较大时，边缘区域。<p>
·某个区域的角点响应函数绝对值较小时，平坦区域。<p>

## 4.算法流程
**·将图像转为灰度图像**<p>
&emsp;&emsp;BGR2GRAY函数将彩色转为灰色，执行颜色通道加权平均，将彩色图像的三个通道（蓝色、绿色、红色）加权平均，以计算灰度值。（0.2126、0.7152、0.0722）是用于将不同通道的颜色值转换为灰度的系数。<p>
**·利用Sobel滤波器求出海森矩阵<p>**
&emsp;&emsp;sobel滤波器常用来提取灰度图像的水平边缘（水平特征）和竖直边缘（竖直特征）<p>
&emsp;&emsp;纵向算子，提取图像水平边缘：$\left[\begin{array}{cc}1 & 2 & 1 \\ 0 & 0 & 0 \\-1 & -2 & -1\end{array}\right]$<p>
&emsp;&emsp;横向算子，提取图像竖直边缘：$\left[\begin{array}{cc}1 & 0 & -1 \\ 2 & 0 & -2 \\1 & 0 & -1\end{array}\right]$<p>
**·将高斯滤波器作用于$I_x^2 $、$I_y^2 $、$I_x I_y$**<p>
 &emsp;&emsp;${e}^{-\frac{{\delta x}^2+{\delta y}^2}{2{\sigma}^2}}$计算了高斯权重，这个权重表示了高斯滤波器的响应，离核中心越远的位置权重越小;之后要对权重矩阵归一化处理一下。<p>
**·计算每个像素的R并根据R值取角点**<p>

In [1]:
import cv2
import numpy as np

#定义函数
def BGR2GRAY(img):
    gray = 0.2126 * img[...,2] + 0.7152 * img[...,1] + 0.0722 * img[...,0]
    gray=gray.astype(np.uint8)
    return gray

def Sobel_filter(img):
    sobely = np.array(((1, 2, 1),(0, 0, 0),(-1, -2, -1)))
    sobelx = np.array(((1, 0, -1),(2, 0, -2),(1, 0, -1)))
    
    Ix = np.zeros_like(img, dtype = np.float32)
    Iy = np.zeros_like(img, dtype = np.float32)
    
    H ,W = img.shape 
    tmp = np.pad(img,(1, 1),'edge')
    
    for y in range(H):
        for x in range(W):
            Iy[y, x] = np.mean(tmp[y : y + 3, x : x + 3] * sobelx)
            Ix[y, x] = np.mean(tmp[y : y + 3, x : x + 3] * sobely)
    Ix2 = Ix ** 2
    Iy2 = Iy ** 2
    Ixy = Ix * Iy
    return Ix2, Iy2, Ixy

def Guassian_filter(I, ksize = 3, sigma = 3):
    #Guassian Kernel
    k = np.zeros((ksize, ksize), dtype = np.float32)
    for y in range(ksize):
        for x in range(ksize):
            d_x = x - ksize // 2
            d_y = y - ksize // 2
            k[y,x] = np.exp(-(d_x ** 2 + d_y ** 2)/(2 * sigma ** 2))
            sum_k = np.sum(k)
            k /= sum_k

    tmp = np.pad(I,(ksize // 2, ksize // 2), 'edge')
    H,W = I.shape
    for y in range(H):
        for x in range(W):
            I[y,x] = np.sum(tmp[y : y + ksize, x : x + ksize] * k )
    return I

def Corner_detect(gray, Ix2, Iy2, Ixy, k, th):
    out = np.array((gray, gray, gray))
    out = np.transpose(out,(1, 2, 0))
    
    R = (Ix2 * Iy2 - Ixy ** 2) - k * (Ix2 + Iy2) ** 2
    thres = th * np.max(R)
    out[R >= thres] = [0, 0, 255]
    out = out.astype(np.uint8)
    return out
    
#读入图像并灰度化
filename = 'chessboard2.png'
img = cv2.imread(filename)
gray = BGR2GRAY(img)

#Sobel滤波器求海森矩阵
Ix2, Iy2, Ixy = Sobel_filter(gray)

#高斯滤波器
Ix2 = Guassian_filter(Ix2)
Iy2 = Guassian_filter(Iy2)
Ixy = Guassian_filter(Ixy)

#角点检测
output = Corner_detect(gray, Ix2, Iy2, Ixy, k = 0.04, th = 0.01)

cv2.imshow('Harris Corner Detection',output)
cv2.waitKey(0)

-1

**遇到的问题分析**<p>
&emsp;&emsp;在定义角点检测函数Corner_detect时，需要将图片信息赋给out变量。但是起初把gray赋给out时会报错“NumPy boolean array indexing assignment cannot assign 3 input values to the 1193 output values where the mask is true”，gray是二维灰度图像，预期输出的out是一个三通道图像，不匹配。<p>
&emsp;&emsp;故需要将（gray，gray，gray）的三个相同的灰度图像组成的三通道图像赋给out。但又报错了“boolean index did not match indexed array along dimension 0; dimension is 3 but corresponding boolean dimension is 799”，原因是没有np.transpose(out,(1, 2, 0))；并且，重新排列轴的顺序不能更改。为什么？

In [21]:
import numpy as np

#正常照片&转换成矩阵形式后
img = cv2.imread('chessboard2.png')
print("img shape:",img.shape)
img_arr = np.array(img)
print("img_array shape:",img_arr.shape)

#由灰度图转换后
gray = 0.2126 * img[...,2] + 0.7152 * img[...,1] + 0.0722 * img[...,0]
out = np.array((gray, gray, gray))
print("out shape:",out.shape)

img shape: (799, 1129, 3)
img_array shape: (799, 1129, 3)
out shape: (3, 799, 1129)


注意：<p>
&emsp;&emsp;transpose和reshape的区别:<p>
·transpose转置操作<p>
·reshape改变形状但元素排布顺序不变

In [ ]:
##直接调用cv2中的cornerHarris
import cv2
import numpy as np
 
#读入图像并转化为float类型，用于传递给harris函数
filename = 'chessboard2.png'
img = cv2.imread(filename)

#灰度化
gray_img = cv2.cvtColor(img, cv2.COLOR_BGR2GRAY)
gray_img = np.float32(gray_img)

#对图像执行harris
blocksize = 2
ksize = 3
k = 0.04
Harris_detector = cv2.cornerHarris(gray_img, blocksize, ksize, k)
 
#膨胀harris结果
dst = cv2.dilate(Harris_detector, None)
 
# 设置阈值
thres = 0.01*dst.max()

#角点展示
img[dst > thres] = [0,0,255]
 
cv2.imshow('Harris Result', img)
 
cv2.waitKey(0)

检测边缘

In [12]:
import cv2
import numpy as np

#定义函数
def BGR2GRAY(img):
    gray = 0.2126 * img[...,2] + 0.7152 * img[...,1] + 0.0722 * img[...,0]
    gray=gray.astype(np.uint8)
    return gray

def Sobel_filter(img):
    sobely = np.array(((1, 2, 1),(0, 0, 0),(-1, -2, -1)))
    sobelx = np.array(((1, 0, -1),(2, 0, -2),(1, 0, -1)))
    
    Ix = np.zeros_like(img, dtype = np.float32)
    Iy = np.zeros_like(img, dtype = np.float32)
    
    H ,W = img.shape 
    tmp = np.pad(img,(1, 1),'edge')
    
    for y in range(H):
        for x in range(W):
            Iy[y, x] = np.mean(tmp[y : y + 3, x : x + 3] * sobelx)
            Ix[y, x] = np.mean(tmp[y : y + 3, x : x + 3] * sobely)
    Ix2 = Ix ** 2
    Iy2 = Iy ** 2
    Ixy = Ix * Iy
    return Ix2, Iy2, Ixy

def Guassian_filter(I, ksize = 3, sigma = 3):
    #Guassian Kernel
    k = np.zeros((ksize, ksize), dtype = np.float32)
    for y in range(ksize):
        for x in range(ksize):
            d_x = x - ksize // 2
            d_y = y - ksize // 2
            k[y,x] = np.exp(-(d_x ** 2 + d_y ** 2)/(2 * sigma ** 2))
            sum_k = np.sum(k)
            k /= sum_k

    tmp = np.pad(I,(ksize // 2, ksize // 2), 'edge')
    H,W = I.shape
    for y in range(H):
        for x in range(W):
            I[y,x] = np.sum(tmp[y : y + ksize, x : x + ksize] * k )
    return I

def Corner_detect(img, Ix2, Iy2, Ixy, k, th):
    out = img
    
    R = (Ix2 * Iy2 - Ixy ** 2) - k * (Ix2 + Iy2) ** 2
    thres = -th * np.max(R)
    out[R <= thres] = [0, 0, 255]
    out = out.astype(np.uint8)
    return out
    
#读入图像并灰度化
filename = 'Obstacle4.jpg'
img = cv2.imread(filename)
gray = BGR2GRAY(img)

#Sobel滤波器求海森矩阵
Ix2, Iy2, Ixy = Sobel_filter(gray)

#高斯滤波器
Ix2 = Guassian_filter(Ix2)
Iy2 = Guassian_filter(Iy2)
Ixy = Guassian_filter(Ixy)

#角点检测
output = Corner_detect(img, Ix2, Iy2, Ixy, k = 0.04, th = 0.01)

cv2.imshow('Harris Corner Detection',output)
cv2.waitKey(0)

-1